In [ ]:
from pyravendb.store import document_store
from entities.user import User
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession\
 .builder\
 .appName("projeto-pmd-consultas")\
 .master("local[*]")\
 .config("spark.jars", "neo4j-connector-apache-spark_2.12-4.1.4_for_spark_3.jar")\
 .getOrCreate()

In [ ]:
with document_store.DocumentStore(
    urls=['http://localhost:8080'],
    database="Projeto-PMD"
) as store:
    store.initialize()

***Recupera um conjunto de usuário que possuem  id na lista fornecida como parametro***

In [ ]:
def get_users(list_ids):
    with store.open_session() as session:
        query_result = list(
            session.query(object_type=User).where_in("id()", list_ids).select("nome", "telefone", "email", "anuncios_feitos")
        )
        return query_result

**Recupera um usuário pelo nome**

In [ ]:
def recupera_usuario(nome):
    with store.open_session() as session:
        return list(session.query(object_type=User).where_equals("nome", nome))

**Cria um Dataframe com os dados do Neo4J obtidos a partir da query anterior**

In [ ]:
def neo4_query(query):
    return (spark.read.format("org.neo4j.spark.DataSource")
            .option("url", "bolt://localhost:11003")
            .option("authentication.basic.username", "neo4j")
            .option("authentication.basic.password", "1234")
            .option("query", query)
            .load())

## **Consultas**

###  **Consulta 1: Consultar o histórico de anúncios dos amigos de um usuário** 

In [ ]:
user_name = 'Béla Tardos'
u = recupera_usuario(user_name)[0]
for a in u.anuncios_recebidos:
    print("Vendedor:", a[1])
    print("Anuncios:")
    print(a[2])
    print("==================================================================================================================================")

### **Consultar os anúncios que os amigos dos meus amigos estão anunciando** 

**Uma query para ser utilizada no Neo4J**

In [ ]:
query_string = "Match (u:User{nome:\"Béla Tardos\"})-[f:FOLLOW*1..2]-(v:User) return distinct v.id"

In [ ]:
df_amigos = neo4_query(query_string)

In [ ]:
df_amigos.show()

In [ ]:
list_r = df_amigos.rdd.map(lambda x: x[0]).collect()

In [ ]:
users_result = get_users(list_r)

In [ ]:
len(users_result)

In [ ]:
for a in users_result:
    print("Vendedor: ", a.nome)
    print("Telefone: ", a.telefone)
    print("Email", a.email)
    print("Anuncios-Feitos:", a.anuncios_feitos)
    print("===================================================================================================================================================================")